In [1]:
from line_solver import *

In [2]:
model = Network('model')
source = Source(model, 'Source')
sink = Sink(model, 'Sink')

# Create 7 places
P = []
for i in range(7):
    P.append(Place(model, f'P{i+1}'))

# Create 8 transitions
T = []
for i in range(8):
    T.append(Transition(model, f'T{i+1}'))

jobclass = OpenClass(model, 'Class1', 0)
source.set_arrival(jobclass, Exp.fitMean(1.0))

In [3]:
# T1 - Timed transition with exponential distribution
mode1 = T[0].add_mode('Mode1')
T[0].set_number_of_servers(mode1, GlobalConstants.MaxInt)
T[0].setDistribution(mode1, Exp(4))
T[0].setEnablingConditions(mode1, jobclass, P[0], 1)
T[0].setFiringOutcome(mode1, jobclass, P[1], 1)

# T2 - Immediate transition
mode2 = T[1].add_mode('Mode1')
T[1].set_number_of_servers(mode2, GlobalConstants.MaxInt)
T[1].setEnablingConditions(mode2, jobclass, P[1], 1)
T[1].setFiringOutcome(mode2, jobclass, P[2], 1)
T[1].setTimingStrategy(mode2, TimingStrategy.IMMEDIATE)
T[1].setFiringPriorities(mode2, 1)
T[1].setFiringWeights(mode2, 1.0)

# T3 - Immediate transition
mode3 = T[2].add_mode('Mode1')
T[2].set_number_of_servers(mode3, GlobalConstants.MaxInt)
T[2].setEnablingConditions(mode3, jobclass, P[1], 1)
T[2].setFiringOutcome(mode3, jobclass, P[3], 1)
T[2].setTimingStrategy(mode3, TimingStrategy.IMMEDIATE)
T[2].setFiringPriorities(mode3, 1)
T[2].setFiringWeights(mode3, 0.6)

# T4 - Immediate transition with multiple enabling/firing conditions
mode4 = T[3].add_mode('Mode1')
T[3].set_number_of_servers(mode4, GlobalConstants.MaxInt)
T[3].setEnablingConditions(mode4, jobclass, P[2], 1)
T[3].setEnablingConditions(mode4, jobclass, P[4], 1)
T[3].setFiringOutcome(mode4, jobclass, P[4], 1)
T[3].setFiringOutcome(mode4, jobclass, P[5], 1)
T[3].setTimingStrategy(mode4, TimingStrategy.IMMEDIATE)
T[3].setFiringPriorities(mode4, 1)

# T5 - Immediate transition with inhibiting conditions
mode5 = T[4].add_mode('Mode1')
T[4].set_number_of_servers(mode5, GlobalConstants.MaxInt)
T[4].setEnablingConditions(mode5, jobclass, P[3], 1)
T[4].setEnablingConditions(mode5, jobclass, P[4], 1)
T[4].setFiringOutcome(mode5, jobclass, P[6], 1)
T[4].setInhibitingConditions(mode5, jobclass, P[5], 1)  # Inhibited by P6
T[4].setTimingStrategy(mode5, TimingStrategy.IMMEDIATE)
T[4].setFiringPriorities(mode5, 1)

# T6 - Timed transition with Erlang distribution
mode6 = T[5].add_mode('Mode1')
T[5].set_number_of_servers(mode6, GlobalConstants.MaxInt)
T[5].setDistribution(mode6, Erlang(2, 2))
T[5].setEnablingConditions(mode6, jobclass, P[5], 1)
T[5].setFiringOutcome(mode6, jobclass, P[0], 1)

# T7 - Timed transition with multiple firing outcomes
mode7 = T[6].add_mode('Mode1')
T[6].set_number_of_servers(mode7, GlobalConstants.MaxInt)
T[6].setDistribution(mode7, Exp(2))
T[6].setEnablingConditions(mode7, jobclass, P[6], 1)
T[6].setFiringOutcome(mode7, jobclass, P[0], 1)
T[6].setFiringOutcome(mode7, jobclass, P[4], 1)

# T8 - Timed transition to sink
mode8 = T[7].add_mode('Mode1')
T[7].set_number_of_servers(mode8, GlobalConstants.MaxInt)
T[7].setDistribution(mode8, Exp(2))
T[7].setEnablingConditions(mode8, jobclass, P[3], 1)
T[7].setFiringOutcome(mode8, jobclass, sink, 1)

In [4]:
# Set up routing matrix
routingMatrix = model.init_routing_matrix()

# Source routing
routingMatrix.set(jobclass, jobclass, source, P[0], 1.0)

# Place to transition routing
routingMatrix.set(jobclass, jobclass, P[0], T[0], 1.0)  # P1 -> T1
routingMatrix.set(jobclass, jobclass, P[1], T[1], 1.0)  # P2 -> T2
routingMatrix.set(jobclass, jobclass, P[1], T[2], 1.0)  # P2 -> T3
routingMatrix.set(jobclass, jobclass, P[2], T[3], 1.0)  # P3 -> T4
routingMatrix.set(jobclass, jobclass, P[3], T[4], 1.0)  # P4 -> T5
routingMatrix.set(jobclass, jobclass, P[4], T[3], 1.0)  # P5 -> T4
routingMatrix.set(jobclass, jobclass, P[4], T[4], 1.0)  # P5 -> T5
routingMatrix.set(jobclass, jobclass, P[5], T[4], 1.0)  # P6 -> T5
routingMatrix.set(jobclass, jobclass, P[5], T[5], 1.0)  # P6 -> T6
routingMatrix.set(jobclass, jobclass, P[6], T[6], 1.0)  # P7 -> T7
routingMatrix.set(jobclass, jobclass, P[3], T[7], 1.0)  # P4 -> T8

# Transition to place/sink routing
routingMatrix.set(jobclass, jobclass, T[0], P[1], 1.0)  # T1 -> P2
routingMatrix.set(jobclass, jobclass, T[1], P[2], 1.0)  # T2 -> P3
routingMatrix.set(jobclass, jobclass, T[2], P[3], 1.0)  # T3 -> P4
routingMatrix.set(jobclass, jobclass, T[3], P[4], 1.0)  # T4 -> P5
routingMatrix.set(jobclass, jobclass, T[3], P[5], 1.0)  # T4 -> P6
routingMatrix.set(jobclass, jobclass, T[4], P[6], 1.0)  # T5 -> P7
routingMatrix.set(jobclass, jobclass, T[5], P[0], 1.0)  # T6 -> P1
routingMatrix.set(jobclass, jobclass, T[6], sink, 1.0)  # T7 -> Sink
routingMatrix.set(jobclass, jobclass, T[6], P[0], 1.0)  # T7 -> P1
routingMatrix.set(jobclass, jobclass, T[6], P[4], 1.0)  # T7 -> P5
routingMatrix.set(jobclass, jobclass, T[7], sink, 1.0)  # T8 -> Sink

model.link(routingMatrix)

In [5]:
# Set initial state
P[0].setState([2])        # 2 tokens at P1
P[1].setState([0])   # 0 tokens at P2
P[2].setState([0])   # 0 tokens at P3
P[3].setState([0])   # 0 tokens at P4
P[4].setState([1])    # 1 token at P5
P[5].setState([0])   # 0 tokens at P6
P[6].setState([0])   # 0 tokens at P7

print("Initial state set:")
print("P1: 2 tokens, P2-P4: 0 tokens, P5: 1 token, P6-P7: 0 tokens")

Initial state set:
P1: 2 tokens, P2-P4: 0 tokens, P5: 1 token, P6-P7: 0 tokens


In [6]:
# Solve the model
solver = SolverJMT(model, seed=23000)
avgTable = solver.get_avg_table()

JMT Model: /tmp/workspace/jsim/428458131443126086/jmodel.jsim
JMT [method: default, lang: java, env: 21.0.8] completed in 3.992487s.
  Station JobClass    QLen  Util   RespT  ResidT    ArvR    Tput
0  Source   Class1  0.0000   0.0  0.0000  0.0000  0.0000  1.0161
1      P1   Class1  0.8524   0.0  0.2550  0.4911  3.3732  3.3769
2      P2   Class1  0.0000   0.0  0.0000  0.0000  3.3769  3.3769
3      P3   Class1  0.0995   0.0  0.0451  0.0434  2.1128  2.1129
4      P4   Class1  0.5152   0.0  0.4132  0.1990  1.2609  1.2589
5      P5   Class1  0.8908   0.0  0.3802  0.2535  2.3421  2.3432
6      P6   Class1  2.0874   0.0  0.9921  0.4042  2.1129  2.1083
7      P7   Class1  0.1124   0.0  0.4965  0.7724  0.2279  0.2278
